In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from pandas.testing import assert_frame_equal
import matplotlib.pyplot as plt
import seaborn as sns
p1 = Path.cwd() / 'back_data'

In [2]:
college = pd.read_csv(p1 / 'college.csv')
columns = college.columns

In [5]:
# index 형식(columns 포함)은 values 속성을 사용해 기저 numpy 배열에 접근 가능
# 스칼라, 리스트, 슬라이스로 아이템 선택 가능
# 기본 산술과 비교 연산자 적용 가능
# index 형식은 immutable -> 값을 변경하려면 리스트로 전환해야
# index 형식은 set와 유사(단 중복값 지원) -> 합집합, 교집합, 차집합, 대칭차집합 지원
columns + '_A'

Index(['INSTNM_A', 'CITY_A', 'STABBR_A', 'HBCU_A', 'MENONLY_A', 'WOMENONLY_A',
       'RELAFFIL_A', 'SATVRMID_A', 'SATMTMID_A', 'DISTANCEONLY_A', 'UGDS_A',
       'UGDS_WHITE_A', 'UGDS_BLACK_A', 'UGDS_HISP_A', 'UGDS_ASIAN_A',
       'UGDS_AIAN_A', 'UGDS_NHPI_A', 'UGDS_2MOR_A', 'UGDS_NRA_A',
       'UGDS_UNKN_A', 'PPTUG_EF_A', 'CURROPER_A', 'PCTPELL_A', 'PCTFLOAN_A',
       'UG25ABV_A', 'MD_EARN_WNE_P10_A', 'GRAD_DEBT_MDN_SUPP_A'],
      dtype='object')

In [6]:
# index 정렬 과정에서 양 쪽 index가 동일하지 않을 경우 카티션 곱 생성
# index가 고유하거나 둘 다 정확히 동일한 요소 및 순서를 가질 경우 카디션 곱 미생성
# index가 고유하지 않을 경우 요소는 동일하나 순서가 다르면 카티션 곱 생성
# 여러 열로 groupby 작업을 하거나 열 중 하나가 categorical 형식인 경우 카티션 곱 생성 -> observed=True 인자 활용 필요

In [7]:
# 리스트 언패킹과 list comprehension을 활용하여 3개 baseball 데이터셋 호출
bball14, bball15, bball16 = [pd.read_csv(p1 / f'baseball{year}.csv', index_col='playerID') for year in [14, 15, 16]]

In [9]:
# index 형식의 difference() 메서드 활용 -> 차집합의 개념
bball14.index.difference(bball15)

Index(['altuvjo01', 'cartech02', 'castrja01', 'corpoca01', 'dominma01',
       'fowlede01', 'gonzama01', 'grossro01', 'guzmaje01', 'hoeslj01',
       'krausma01', 'marisja01', 'preslal01', 'singljo02', 'springe01',
       'villajo01'],
      dtype='object', name='playerID')

In [12]:
hits14 = bball14['H']
hits15 = bball15['H']
hits16 = bball16['H']
# add() 메서드 활용하여 더하면서 fill_value=0 인자 전달하여 결측치 고려
hits_total = hits14.add(hits15, fill_value=0).add(hits16, fill_value=0)
hits_total.head(), hits_total.hasnans

(playerID
 altuvjo01    641.0
 bregmal01     53.0
 cartech02    193.0
 castrja01    243.0
 congeha01     46.0
 Name: H, dtype: float64,
 False)

In [13]:
# DataFrame이 다른 DataFrame이나 Series에서 새 열을 추가할 때 index가 먼저 정렬된 다음 새 열 생성

In [15]:
employee = pd.read_csv(p1 / 'employee.csv')
dept_sal = employee[['DEPARTMENT', 'BASE_SALARY']]
# sort_values() 메서드 활용하여 정렬
dept_sal = dept_sal.sort_values(by=['DEPARTMENT', 'BASE_SALARY'], ascending=[True, False])

In [20]:
# DEPARTMENT의 중복값을 제거하여 가장 큰 BASE_SALARY만 남기기 -> DEPARTMENT를 index로 설정
max_dept_sal = dept_sal.drop_duplicates(subset='DEPARTMENT').set_index('DEPARTMENT')
max_dept_sal.head()

,BASE_SALARY
DEPARTMENT,
Admn. & Regulatory Affairs,140416.0
City Controller's Office,64251.0
City Council,100000.0
Convention and Entertainment,38397.0
Dept of Neighborhoods (DON),89221.0


In [21]:
# index가 자동 정렬되므로, max_dept_sal을 기존 DataFrame에 추가하면 잘 삽입
# but DEPARTMENT에 중복된 부서가 있었다면 카티션 곱이 생성되었을 것
(employee
.set_index('DEPARTMENT')
.assign(MAX_DEPT_SALARY=max_dept_sal['BASE_SALARY'])
.head()
)

,UNIQUE_ID,POSITION_TITLE,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE,MAX_DEPT_SALARY
DEPARTMENT,,,,,,,,,,
Municipal Courts Department,0,ASSISTANT DIRECTOR (EX LVL),121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13,121862.0
Library,1,LIBRARY ASSISTANT,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18,107763.0
Houston Police Department-HPD,2,POLICE OFFICER,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03,199596.0
Houston Fire Department (HFD),3,ENGINEER/OPERATOR,63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25,210588.0
General Services Department,4,ELECTRICIAN,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22,89194.0


In [28]:
# groupby의 transform() 메서드 활용하여 복제 가능
max_sal = (employee
.groupby('DEPARTMENT')['BASE_SALARY']
# transform('max')를 통해 기존 인덱스를 유지하며 BASE_SALARY의 최대값 추출
.transform('max')
)
max_sal

0       121862.0
1       107763.0
2       199596.0
3       210588.0
4        89194.0
          ...   
1995    199596.0
1996    210588.0
1997    199596.0
1998    199596.0
1999    210588.0
Name: BASE_SALARY, Length: 2000, dtype: float64

In [30]:
(employee
# rename() 메서드를 통해 max_sal Series의 이름 지정. 인덱스를 기준으로 합치므로 left_index, right_index에 True 지정
.merge(max_sal.rename('MAX_DEPT_SALARY'), how='inner', left_index=True, right_index=True)
.head()
)

,UNIQUE_ID,POSITION_TITLE,DEPARTMENT,BASE_SALARY,RACE,EMPLOYMENT_TYPE,GENDER,EMPLOYMENT_STATUS,HIRE_DATE,JOB_DATE,MAX_DEPT_SALARY
0,0,ASSISTANT DIRECTOR (EX LVL),Municipal Courts Department,121862.0,Hispanic/Latino,Full Time,Female,Active,2006-06-12,2012-10-13,121862.0
1,1,LIBRARY ASSISTANT,Library,26125.0,Hispanic/Latino,Full Time,Female,Active,2000-07-19,2010-09-18,107763.0
2,2,POLICE OFFICER,Houston Police Department-HPD,45279.0,White,Full Time,Male,Active,2015-02-03,2015-02-03,199596.0
3,3,ENGINEER/OPERATOR,Houston Fire Department (HFD),63166.0,White,Full Time,Male,Active,1982-02-08,1991-05-25,210588.0
4,4,ELECTRICIAN,General Services Department,56347.0,White,Full Time,Male,Active,1989-06-19,1994-10-22,89194.0


In [31]:
college = pd.read_csv(p1 / 'college.csv', index_col='INSTNM')

In [34]:
# 'MD_EARN_WNE_P10', 'GRAD_DEBT_MDN_SUPP' 열의 경우 수치가 되면 좋지만 object 형식이므로 바꿀 필요
# value_counts() 메서드 활용하여 빈도 확인하면서 문제점 파악 -> PrivacySuppressed가 다수
cols = ['MD_EARN_WNE_P10', 'GRAD_DEBT_MDN_SUPP']
[college[col].value_counts() for col in cols]

[PrivacySuppressed    822
 38800                151
 21500                 97
 49200                 78
 27400                 46
                     ... 
 84000                  1
 66900                  1
 52800                  1
 67800                  1
 186500                 1
 Name: MD_EARN_WNE_P10, Length: 598, dtype: int64,
 PrivacySuppressed    1510
 9500                  514
 27000                 306
 25827.5               136
 25000                 124
                      ... 
 9604                    1
 19262                   1
 8099                    1
 8050                    1
 11061                   1
 Name: GRAD_DEBT_MDN_SUPP, Length: 2038, dtype: int64]

In [36]:
# pd.to_numeric() 함수 사용 가능 -> replace()로 바꾸고 형변환 하는 것 보다 이게 더 효율적
for col in cols:
    # errors 파라미터 활용 -> 'coerce', 'ignore', 'raise' 활용 가능
    college[col] = pd.to_numeric(college[col], errors='coerce')
# dtypes 속성을 통해 변화 확인
college.loc[:, cols].dtypes

MD_EARN_WNE_P10       float64
GRAD_DEBT_MDN_SUPP    float64
dtype: object

In [37]:
college_n = college.select_dtypes('number')
college_n.head()

,HBCU,MENONLY,WOMENONLY,RELAFFIL,SATVRMID,SATMTMID,DISTANCEONLY,UGDS,UGDS_WHITE,UGDS_BLACK,...,UGDS_2MOR,UGDS_NRA,UGDS_UNKN,PPTUG_EF,CURROPER,PCTPELL,PCTFLOAN,UG25ABV,MD_EARN_WNE_P10,GRAD_DEBT_MDN_SUPP
INSTNM,,,,,,,,,,,,,,,,,,,,,
Alabama A & M University,1.0,0.0,0.0,0,424.0,420.0,0.0,4206.0,0.0333,0.9353,...,0.0000,0.0059,0.0138,0.0656,1,0.7356,0.8284,0.1049,30300.0,33888.0
University of Alabama at Birmingham,0.0,0.0,0.0,0,570.0,565.0,0.0,11383.0,0.5922,0.2600,...,0.0368,0.0179,0.0100,0.2607,1,0.3460,0.5214,0.2422,39700.0,21941.5
Amridge University,0.0,0.0,0.0,1,NaN,NaN,1.0,291.0,0.2990,0.4192,...,0.0000,0.0000,0.2715,0.4536,1,0.6801,0.7795,0.8540,40100.0,23370.0
University of Alabama in Huntsville,0.0,0.0,0.0,0,595.0,590.0,0.0,5451.0,0.6988,0.1255,...,0.0172,0.0332,0.0350,0.2146,1,0.3072,0.4596,0.2640,45500.0,24097.0
Alabama State University,1.0,0.0,0.0,0,425.0,430.0,0.0,4811.0,0.0158,0.9208,...,0.0098,0.0243,0.0137,0.0892,1,0.7347,0.7554,0.1270,26600.0,33118.5
